In [20]:
# combine pred into different pred_x folders
# combine heuristics - 

import os
import json
import glob
from shutil import copyfile, rmtree


reex_dir = '/home/locobotm/0512_data/reexplore_final_2/'
base_dir = '/home/locobotm/0512_data/reexplore_final_2/baseline_explore/'


heuristics = ['c1pp', 's1pp', 'r1', 'r2']
prop_range = range(0, 18, 4)


def copyover(in_dir, out_dir, p):
    os.makedirs(out_dir, exist_ok=True)
    for x in range(p):
        if os.path.isfile(os.path.join(in_dir, 'pred', f'{x:05d}.npy')):
            copyfile(os.path.join(in_dir, 'rgb', f'{x:05d}.jpg'), os.path.join(out_dir, 'rgb', f'{x+1:05d}.jpg'))
            copyfile(os.path.join(in_dir, 'pred', f'{x:05d}.npy'), os.path.join(out_dir, 'seg', f'{x+1:05d}.npy'))
    
# def copy_og_label(base_dir, reex_json, obj_id, out_pred_dir):
#     with open(reex_json, 'r') as f:
#         data = json.load(f)
#     src_img_id = f"{int(data[obj_id]['src_img_id']):05d}"
#     dst_img_id = f'{0:05d}'
    
#     copyfile(os.path.join(base_dir, 'rgb', f'{src_img_id}.jpg'), os.path.join(out_pred_dir, 'rgb', f'{dst_img_id}.jpg'))
#     copyfile(os.path.join(base_dir, 'seg', f'{src_img_id}.npy'), os.path.join(out_pred_dir, 'seg', f'{dst_img_id}.npy'))

def copy_og_label(base_dir, src_img_id, out_pred_dir):
    dst_img_id = f'{0:05d}'    
    copyfile(os.path.join(base_dir, 'rgb', f'{src_img_id:05d}.jpg'), os.path.join(out_pred_dir, 'rgb', f'{dst_img_id}.jpg'))
    copyfile(os.path.join(base_dir, 'seg', f'{src_img_id:05d}.npy'), os.path.join(out_pred_dir, 'seg', f'{dst_img_id}.npy'))

label_to_og_frames = {
    'armchair': [142, 140, 88],
    'plant': [88, 87, 90],
    'hamper': [103, 99, 96]
}

reex_id_to_label = {
    0: 'armchair',
    2: 'plant',
    4: 'hamper',
}

for sample in range(3):
    for obj in ['0', '2', '4']: #os.listdir(reex_dir):
        if obj.isdigit():
            src_img_id = label_to_og_frames[reex_id_to_label[int(obj)]][sample]
            for h in heuristics:
                heu_dir = os.path.join(reex_dir, obj, h)
                output_dir = os.path.join(reex_dir, obj, h, f'propagated_{sample}')
                # print(f'output_dir {output_dir}')
                if os.path.isdir(output_dir):
                    # print(f'rmtree {output_dir}')
                    rmtree(output_dir)
                os.makedirs(output_dir, exist_ok=True)

                for p in prop_range:
                    out_pred_dir = os.path.join(output_dir, f'pred_label_p{p}')

                    # make seg and rgb dir, all we need for coco + train
                    os.makedirs(out_pred_dir, exist_ok=True)
                    os.makedirs(os.path.join(out_pred_dir, 'rgb'), exist_ok=True)
                    os.makedirs(os.path.join(out_pred_dir, 'seg'), exist_ok=True)

                    # copy_og_label(base_dir, os.path.join(reex_dir, 'reexplore_data.json'), obj, out_pred_dir)
                    copy_og_label(base_dir, src_img_id, out_pred_dir)
                    if p > 0:
                        copyover(heu_dir, out_pred_dir, p)
                        act_p = len(os.listdir(os.path.join(out_pred_dir, 'seg')))
                        # assert act_p == p+1, f"prop length wrong! expected {p+1}, got {act_p}, out {out_pred_dir}"
                        if p+1 != act_p:
                            print(f"prop length wrong! expected {p+1}, got {act_p}, out {out_pred_dir}")

prop length wrong! expected 9, got 6, out /home/locobotm/0512_data/reexplore_final_2/2/s1pp/propagated_0/pred_label_p8
prop length wrong! expected 13, got 6, out /home/locobotm/0512_data/reexplore_final_2/2/s1pp/propagated_0/pred_label_p12
prop length wrong! expected 17, got 6, out /home/locobotm/0512_data/reexplore_final_2/2/s1pp/propagated_0/pred_label_p16
prop length wrong! expected 9, got 6, out /home/locobotm/0512_data/reexplore_final_2/2/s1pp/propagated_1/pred_label_p8
prop length wrong! expected 13, got 6, out /home/locobotm/0512_data/reexplore_final_2/2/s1pp/propagated_1/pred_label_p12
prop length wrong! expected 17, got 6, out /home/locobotm/0512_data/reexplore_final_2/2/s1pp/propagated_1/pred_label_p16
prop length wrong! expected 9, got 6, out /home/locobotm/0512_data/reexplore_final_2/2/s1pp/propagated_2/pred_label_p8
prop length wrong! expected 13, got 6, out /home/locobotm/0512_data/reexplore_final_2/2/s1pp/propagated_2/pred_label_p12
prop length wrong! expected 17, got 6,

In [30]:
from pathlib import Path               
            
# combine everything 3 * 5 models
combinations = {
    'c1ppr2': ['c1pp', 'r2'],
    'c1pps1pp': ['c1pp', 's1pp'],
    's1ppr2': ['s1pp', 'r2'],
    'r1r2': ['r1', 'r2'],
}

# combinations = {
#     'c1pp': ['c1pp'],
#     's1pp': ['s1pp'],
#     'r1': ['r1'],
# }

def lookup(h):
    ans = []
    for k, v in combinations.items():
        if h in v:
            ans.append(k)
    return ans
            
def copyappend(in_dir, out_dir):
    ctr = len(os.listdir(os.path.join(out_dir, 'seg')))
    
    # copy rgb, seg
    for x in os.listdir(os.path.join(in_dir, 'seg')):
        x = x.split('.')[0]
        copyfile(os.path.join(in_dir, 'rgb', f'{x}.jpg'), os.path.join(out_dir, 'rgb', f'{ctr:05d}.jpg'))
        copyfile(os.path.join(in_dir, 'seg', f'{x}.npy'), os.path.join(out_dir, 'seg', f'{ctr:05d}.npy'))
        ctr += 1
        

def copypred(train_data, pred_dir):
    pf = pred_dir.split('/')[-2:]
    dsts = lookup(pred_dir.split('/')[-3])
    # print(f'dsts {dsts}')
    for d in dsts:
        out_dir = os.path.join(train_data, d, '/'.join(pf))
        # print(f'makedirs {out_dir}, pf {pf}')
        if not os.path.isdir(out_dir):
            # print(f'makedirs {out_dir}')
            os.makedirs(out_dir, exist_ok=True)
            os.makedirs(os.path.join(out_dir, 'rgb'), exist_ok=True)
            os.makedirs(os.path.join(out_dir, 'seg'), exist_ok=True)
        
        copyappend(pred_dir, out_dir)

train_data = os.path.join(reex_dir, '../train_data_1805_3sample')
if os.path.isdir(train_data):
    print(f'rmtree {train_data}')
    rmtree(train_data)
        
# for sample in range(3):
    
    
for p in prop_range:
    collect = f'pred_label_p{p}'
    for x in Path(reex_dir).rglob(collect):
        # print(x)
        copypred(train_data, str(x))

# for k, v in combinations.items():
#     comb_out = os.path.join(reex_dir, k)
#     os.path.makedirs(comb_out, exist_ok=True)
#     for h in v:
#         for obj in os.listdir(reex_dir):
#             if obj.isdigit():
                # copy over pred_label_px dir
        

rmtree /home/locobotm/0512_data/reexplore_final_2/../train_data_1805_3sample


In [3]:
?shutil.copyfile

Object `shutil.copyfile` not found.
